# ETL: Turismo

## Librerias

In [1]:
# Generales
import pandas as pd
import numpy as np
import time
import re
import warnings
warnings.filterwarnings('ignore')

# Funciones Snowflake
import funciones as snow_func

In [2]:
# Aumentar número de columnas que se pueden ver
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.10f}'.format
# Cada columna será tan grande como sea necesario para mostrar todo su contenido
pd.set_option('display.max_colwidth', 0)

### Snowflake

In [3]:
# Librerias necesarias para subir a Snowflake
import os
import json
import snowflake.connector # [pip install snowflake-connector-python]
from snowflake.connector.pandas_tools import write_pandas # [pip install "snowflake-connector-python[pandas]"]
from snowflake.snowpark import Session

In [4]:
# Paso 1: Definir la ruta al archivo JSON en el escritorio
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop\Conn")
json_file_path = os.path.join(desktop_path, "snowflake_credentials.json")

# Paso 2: Leer las credenciales desde el archivo JSON
with open(json_file_path, 'r') as file:
    credentials = json.load(file)

# Paso 3: Definir los parámetros de conexión usando las credenciales
connection_parameters = {
        "account": credentials["ACCOUNT_SNOWFLAKE"],
        "user": credentials["USER_SNOWFLAKE"],
        "password": credentials["PASSWORD_SNOWFLAKE"],
        "role": credentials["ROLE_SNOWFLAKE"],
        "warehouse": credentials["WAREHOUSE"]
    }

# Paso 5: Crear un objeto de conexión utilizando snowflake.connector
session = Session.builder.configs(connection_parameters).create()
print("Sesión actual:", {session})

Sesión actual: {<snowflake.snowpark.session.Session object at 0x000001C9A8D0B410>}


In [5]:
# Crear objeto de conexión
conn = session.connection

## 0. Totales de control

In [6]:
# Solo se debe cambiar la ubicación del archivo
path_turismo = "C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/TURISMO/"
turismo_totales_cerrado = 'TOTALES-CERRADO_data.csv'
turismo_totales_corrido = 'TOTALES-CORRIDO_data.csv'

In [7]:
# Control cerrado
df_control_cerrado = pd.read_csv(path_turismo + turismo_totales_cerrado, sep=";", decimal=",")

# Elegir años de interés
df_control_cerrado = df_control_cerrado[
    (df_control_cerrado['Nombres de medidas'] == 2022) | (df_control_cerrado['Nombres de medidas'] == 2023)
]

# Crear el dataframe de control
df_control_cerrado = pd.DataFrame(
    [df_control_cerrado["Valores de medidas"].values], 
    columns=df_control_cerrado["Nombres de medidas"].astype(str).values  # Convertir a cadenas
)

# Renombrar columnas
df_control_cerrado = df_control_cerrado.rename(columns={'2022': 'T_1_YEAR', '2023': 'T_YEAR'})

# Df de control
df_control_cerrado

,T_YEAR,T_1_YEAR
0,4076203,3279931


In [8]:
# Control corrido
df_control_corrido = pd.read_csv(path_turismo + turismo_totales_corrido, sep=";", decimal=",")

# Crear el dataframe de control
df_control_corrido = pd.DataFrame(
    [df_control_corrido["Valores de medidas"].values], 
    columns=df_control_corrido["Nombres de medidas"].astype(str).values  # Convertir a cadenas
)

# Renombrar columnas
df_control_corrido = df_control_corrido.rename(columns={'YTD actual' : 'T_YEAR', 'YTD anterior' : 'T_1_YEAR'})

# Df de control
df_control_corrido

,T_1_YEAR,T_YEAR
0,3323601,3663252


In [9]:
# Crear totales por variable de control

# Cerrado
control_cerrado_t_1 = df_control_cerrado['T_1_YEAR'].sum()
control_cerrado_t = df_control_cerrado['T_YEAR'].sum()

# Corrido
control_corrido_t_1 = df_control_corrido['T_1_YEAR'].sum()
control_corrido_t = df_control_corrido['T_YEAR'].sum()

## 0.1 Dataframe de países

In [10]:
# Importar el dataframe de ST PAISES
df_paises = session.sql("SELECT * FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES").to_pandas()

# Crear dataframe para comparación}
df_paises = df_paises[['CODIGO_PAIS_MIGRACION', 'NOMBRE_PAIS_MIGRACION']].drop_duplicates()

## 1. Turismo cerrado

In [11]:
# Solo se debe cambiar la ubicación del archivo
turismo_cerrado = 'CONSULTA-CERRADO_data.csv'

In [12]:
# Importar datos cerrado de turismo
df_turismo_cerrado = pd.read_csv(path_turismo + turismo_cerrado, sep=";", 
                                 dtype= {'Región país residencia' : str, 
                                        'HUB__C (País residencia) (grupo)' : str,
                                        'COD_TURISMO (DIM_PAIS) #2' : str,
                                        'Análisis extranjeros:' : str, 
                                        'ID_DPTO' : str,
                                        'Departamento' : str, 
                                        'ID_CIUDAD' : str, 
                                        'Ciudad' : str, 
                                        'Género' : str, 
                                        'Motivo viaje Pc' : str,
                                        'Rango Turismo' : str, 
                                        'Nombres de medidas' : str, 
                                        'Valores de medidas' : float})

# Crear df para comparar países en un paso posterior
df_turismo_cerrado_paises = df_turismo_cerrado[['COD_TURISMO (DIM_PAIS) #2', 'Análisis extranjeros:']].drop_duplicates()
                                        
# Determinar las columnas "id" (todas excepto 'Nombres de medidas' y 'Valores de medidas')
id_columns = [
    col for col in df_turismo_cerrado.columns 
    if col not in ['Nombres de medidas', 'Valores de medidas']
]

# Pivotar el DataFrame para que 'Nombres de medidas' sea el encabezado y sumar 'Valores de medidas'
df_turismo_cerrado = pd.DataFrame(
    df_turismo_cerrado
    .groupby(id_columns + ['Nombres de medidas'], as_index=False)['Valores de medidas'].sum()
    .pivot(index=id_columns, columns='Nombres de medidas', values='Valores de medidas').reset_index()
)

# Eliminar columnas que no se usan
columnas_extras = ['Análisis extranjeros:', 'Departamento', 'Ciudad']
df_turismo_cerrado = df_turismo_cerrado.drop(columns=columnas_extras)

# Cambiar nombres de columnas
df_turismo_cerrado = df_turismo_cerrado.rename(columns={'Región país residencia' : 'CONTINENTE', 
                                                        'HUB__C (País residencia) (grupo)' : 'HUB', 
                                                        'COD_TURISMO (DIM_PAIS) #2' : 'PAIS_RESIDENCIA', 
                                                        'ID_DPTO' : 'DPTO_HOSPEDAJE', 
                                                        'ID_CIUDAD' : 'CIUDAD_HOSPEDAJE', 
                                                        'Género' : 'DESCRIPCION_GENERO', 
                                                        'Motivo viaje Pc' : 'MOVC_NOMBRE',
                                                        'Rango Turismo' : 'RANGO_EDAD' ,
                                                        '2022   ' : 'SUMA_TURISMO_T_1', 
                                                        '2023  ' : 'SUMA_TURISMO_T'})

# Eliminar datos con ambos casos en cero, en teoría no hay pero sirve para reducir la dimensionalidad
df_turismo_cerrado = df_turismo_cerrado[~((df_turismo_cerrado['SUMA_TURISMO_T_1'] == 0) & (df_turismo_cerrado['SUMA_TURISMO_T'] == 0))]

# Asegurar que los valores en DPTO_HOSPEDAJE son strings y rellenar con ceros a la izquierda si tienen longitud menor a 2
df_turismo_cerrado['DPTO_HOSPEDAJE'] = df_turismo_cerrado['DPTO_HOSPEDAJE'].astype(str).str.zfill(2)

# Asegurar que los valores en CIUDAD_HOSPEDAJE son strings y rellenar con ceros a la izquierda si tienen longitud menor a 5
df_turismo_cerrado['CIUDAD_HOSPEDAJE'] = df_turismo_cerrado['CIUDAD_HOSPEDAJE'].astype(str).str.zfill(5)

In [13]:
# Verificar totales

# Cerrado t-1
resultado_cerrado_t_1 = df_turismo_cerrado['SUMA_TURISMO_T_1'].sum() == control_cerrado_t_1
# Cerrado t
resultado_cerrado_t = df_turismo_cerrado['SUMA_TURISMO_T'].sum() == control_cerrado_t

# Mostrar resultados: todos deben ser verdaderos
print(resultado_cerrado_t_1, resultado_cerrado_t)

True True


In [14]:
# Verificar países

# Unir con la consulta de ST PAISES
df_turismo_cerrado_paises = df_turismo_cerrado_paises.merge(df_paises, how='left', left_on='COD_TURISMO (DIM_PAIS) #2', right_on='CODIGO_PAIS_MIGRACION')

# No deben salir países de verdad nulos, solo agrupaciones internacionales
df_turismo_cerrado_paises[df_turismo_cerrado_paises['NOMBRE_PAIS_MIGRACION'].isna()]

,COD_TURISMO (DIM_PAIS) #2,Análisis extranjeros:,CODIGO_PAIS_MIGRACION,NOMBRE_PAIS_MIGRACION


In [15]:
# El df obtenido de Tableau tiene ya las agrupaciones por lo que solo se ordenan las columnas y se crea la agrupación

# ST PAISES CERRADO
ST_PAISES_CERRADO = df_turismo_cerrado

# Crear agrupación
ST_PAISES_CERRADO['AGRUPACION'] = 'PAISES'

# Ordenar
ST_PAISES_CERRADO = ST_PAISES_CERRADO[['AGRUPACION', 'CONTINENTE', 'HUB', 'PAIS_RESIDENCIA', 'DPTO_HOSPEDAJE', 'CIUDAD_HOSPEDAJE', 'DESCRIPCION_GENERO', 'MOVC_NOMBRE', 'RANGO_EDAD', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

## 2. Turismo corrido

In [16]:
# Solo se debe cambiar la ubicación del archivo
turismo_corrido = 'CONSULTA-CORRIDO_data.csv'

In [17]:
# Importar datos corrido de turismo
df_turismo_corrido = pd.read_csv(path_turismo + turismo_corrido, sep=";", 
                                 dtype= {'Región país residencia' : str, 
                                        'HUB__C (País residencia) (grupo)' : str,
                                        'COD_TURISMO (DIM_PAIS) #2' : str,
                                        'Análisis extranjeros:' : str, 
                                        'ID_DPTO' : str,
                                        'Departamento' : str, 
                                        'ID_CIUDAD' : str, 
                                        'Ciudad' : str, 
                                        'Género' : str, 
                                        'Motivo viaje Pc' : str,
                                        'Rango Turismo' : str, 
                                        'Nombres de medidas' : str, 
                                        'Valores de medidas' : float})

# Crear df para comparar países en un paso posterior
df_turismo_corrido_paises = df_turismo_corrido[['COD_TURISMO (DIM_PAIS) #2', 'Análisis extranjeros:']].drop_duplicates()
                                        
# Determinar las columnas "id" (todas excepto 'Nombres de medidas' y 'Valores de medidas')
id_columns = [
    col for col in df_turismo_corrido.columns 
    if col not in ['Nombres de medidas', 'Valores de medidas']
]

# Pivotar el DataFrame para que 'Nombres de medidas' sea el encabezado y sumar 'Valores de medidas'
df_turismo_corrido = pd.DataFrame(
    df_turismo_corrido
    .groupby(id_columns + ['Nombres de medidas'], as_index=False)['Valores de medidas'].sum()
    .pivot(index=id_columns, columns='Nombres de medidas', values='Valores de medidas').reset_index()
)

# Eliminar columnas que no se usan
columnas_extras = ['Análisis extranjeros:', 'Departamento', 'Ciudad']
df_turismo_corrido = df_turismo_corrido.drop(columns=columnas_extras)

# Cambiar nombres de columnas
df_turismo_corrido = df_turismo_corrido.rename(columns={'Región país residencia' : 'CONTINENTE', 
                                                        'HUB__C (País residencia) (grupo)' : 'HUB', 
                                                        'COD_TURISMO (DIM_PAIS) #2' : 'PAIS_RESIDENCIA', 
                                                        'ID_DPTO' : 'DPTO_HOSPEDAJE', 
                                                        'ID_CIUDAD' : 'CIUDAD_HOSPEDAJE', 
                                                        'Género' : 'DESCRIPCION_GENERO', 
                                                        'Motivo viaje Pc' : 'MOVC_NOMBRE',
                                                        'Rango Turismo' : 'RANGO_EDAD' ,
                                                        'YTD anterior' : 'SUMA_TURISMO_T_1', 
                                                        'YTD actual' : 'SUMA_TURISMO_T'})

# Eliminar datos con ambos casos en cero, en teoría no hay pero sirve para reducir la dimensionalidad
df_turismo_corrido = df_turismo_corrido[~((df_turismo_corrido['SUMA_TURISMO_T_1'] == 0) & (df_turismo_corrido['SUMA_TURISMO_T'] == 0))]

# Asegurar que los valores en DPTO_HOSPEDAJE son strings y rellenar con ceros a la izquierda si tienen longitud menor a 2
df_turismo_corrido['DPTO_HOSPEDAJE'] = df_turismo_corrido['DPTO_HOSPEDAJE'].astype(str).str.zfill(2)

# Asegurar que los valores en CIUDAD_HOSPEDAJE son strings y rellenar con ceros a la izquierda si tienen longitud menor a 5
df_turismo_corrido['CIUDAD_HOSPEDAJE'] = df_turismo_corrido['CIUDAD_HOSPEDAJE'].astype(str).str.zfill(5)

In [18]:
# Verificar totales

# Corrido t-1
resultado_corrido_t_1 = df_turismo_corrido['SUMA_TURISMO_T_1'].sum() == control_corrido_t_1
# Corrido t
resultado_corrido_t = df_turismo_corrido['SUMA_TURISMO_T'].sum() == control_corrido_t

# Mostrar resultados: todos deben ser verdaderos
print(resultado_corrido_t_1, resultado_corrido_t)

True True


In [19]:
# Verificar países

# Unir con la consulta de ST PAISES
df_turismo_corrido_paises = df_turismo_corrido_paises.merge(df_paises, how='left', left_on='COD_TURISMO (DIM_PAIS) #2', right_on='CODIGO_PAIS_MIGRACION')

# No deben salir países de verdad nulos, solo agrupaciones internacionales
df_turismo_corrido_paises[df_turismo_corrido_paises['NOMBRE_PAIS_MIGRACION'].isna()]

,COD_TURISMO (DIM_PAIS) #2,Análisis extranjeros:,CODIGO_PAIS_MIGRACION,NOMBRE_PAIS_MIGRACION


In [20]:
# El df obtenido de Tableau tiene ya las agrupaciones por lo que solo se ordenan las columnas y se crea la agrupación

# ST PAISES CORRIDO
ST_PAISES_CORRIDO = df_turismo_corrido

# Crear agrupación
ST_PAISES_CORRIDO['AGRUPACION'] = 'PAISES'

# Ordenar
ST_PAISES_CORRIDO = ST_PAISES_CORRIDO[['AGRUPACION', 'CONTINENTE', 'HUB', 'PAIS_RESIDENCIA', 'DPTO_HOSPEDAJE', 'CIUDAD_HOSPEDAJE', 'DESCRIPCION_GENERO', 'MOVC_NOMBRE', 'RANGO_EDAD', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

## 3. Conectividad

In [21]:
# Solo se debe cambiar la ubicación del archivo
path_insumos = 'C:/Users/nrivera/OneDrive - PROCOLOMBIA/Documentos/017B-Documentos-Colombia/Cargue/Insumos/TURISMO/Conectividad/'
conectividad_file = 'Conectividad nacional.xlsx'
dict_file = 'Diccionario-Departamentos-Conectividad.xlsx'

In [22]:
# Importar datos
df_conectividad = pd.read_excel(path_insumos + conectividad_file, sheet_name="Hoja1", dtype=str)

In [23]:
# Elegir columnas de interés
df_conectividad = df_conectividad[['Aerolinea', 'Ciudad_Origen', 'Ciudad_destino', 'Frecuencias', 'Semana de análisis', 'Departamento_origen', 'Departamento_destino']]

In [24]:
# Cambiar nombres para snowflake
df_conectividad = df_conectividad.rename(columns={
    'Aerolinea' : 'AEROLINEA', 
    'Ciudad_Origen' : 'CIUDAD_ORIGEN', 
    'Ciudad_destino' : 'CIUDAD_DESTINO', 
    'Frecuencias' : 'FRECUENCIAS',
    'Semana de análisis' : 'SEMANA', 
    'Departamento_origen' :  'DEPARTAMENTO_ORGIEN', 
    'Departamento_destino' : 'DEPARTAMENTO_DESTINO'
})

In [25]:
# Importar datos de nombres
df_dict_nombres = pd.read_excel(path_insumos + dict_file, sheet_name="Hoja1", dtype=str)

In [26]:
# Agregar códigos de nombres departamentos
df_conectividad = pd.merge(df_conectividad, df_dict_nombres, left_on=['DEPARTAMENTO_DESTINO'], right_on=['DEPARTAMENTO'], how='left')
# Cambiar nombres 
df_conectividad = df_conectividad.rename(columns={
    'COD_DIVIPOLA' : 'COD_DIVIPOLA_DEPARTAMENTO_DESTINO'
})
df_conectividad = df_conectividad.drop('DEPARTAMENTO', axis=1)

In [27]:
# Agregar códigos de nombres departamentos
df_conectividad = pd.merge(df_conectividad, df_dict_nombres, left_on=['DEPARTAMENTO_ORGIEN'], right_on=['DEPARTAMENTO'], how='left')
# Cambiar nombres 
df_conectividad = df_conectividad.rename(columns={
    'COD_DIVIPOLA' : 'COD_DIVIPOLA_DEPARTAMENTO_ORGIEN'
})
df_conectividad = df_conectividad.drop('DEPARTAMENTO', axis=1)

## 6. Crear DF de validación

In [28]:
# Se crea una tabla con los siguientes campos:
    # - Agrupación: 'CONTINENTES', 'PAISES', 'HUBS', 'TLCS', 'DEPARTAMENTOS', 'COLOMBIA'
    # - Unidad: se va a usar la misma unidad que en la validación de exportaciones para facilitar el proceso de comparación
    # - Tipo periodo: 'TURISMO-CERRADO' O 'TURISMO-CORRIDO'
    # - SUMA_TURISMO_T_1 Y SUMA_TURISMO_T

In [29]:
##########
# COLOMBIA
##########

# Cerrado
df_control_cerrado['TIPO_PERIODO'] = 'TURISMO-CERRADO'

# Corrido
df_control_corrido['TIPO_PERIODO'] = 'TURISMO-CORRIDO'

# DF de control
ST_CONTROL_COLOMBIA = pd.DataFrame(pd.concat([df_control_cerrado, df_control_corrido]))

# Columnas extras
ST_CONTROL_COLOMBIA['UNIDAD'] = 'COLOMBIA'
ST_CONTROL_COLOMBIA['AGRUPACION'] = 'COLOMBIA'

# Ordenar columnas
ST_CONTROL_COLOMBIA = ST_CONTROL_COLOMBIA[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'T_1_YEAR', 'T_YEAR']]
ST_CONTROL_COLOMBIA = ST_CONTROL_COLOMBIA.rename(columns={'T_1_YEAR' : 'SUMA_TURISMO_T_1', 'T_YEAR' : 'SUMA_TURISMO_T'})

In [30]:
###############
# Departamentos
###############

# Archivo departamentos
departamento_control = 'DEPARTAMENTOS_data.csv'
df_control_departamentos = pd.read_csv(path_turismo + departamento_control, sep=";", decimal=",", dtype={'ID_DPTO':str, 'Departamento':str})


# Elegir años de interés

# Cerrado
df_control_departamentos_cerrado = df_control_departamentos[
    (df_control_departamentos['Nombres de medidas'] == '2022   ') | 
    (df_control_departamentos['Nombres de medidas'] == '2023  ') ]

# Pivot
df_cerrado = pd.DataFrame(df_control_departamentos_cerrado.pivot_table(
    index=['ID_DPTO', 'Departamento'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_cerrado.columns.name = None

# Renombrar columnas para el caso Cerrado
df_cerrado.rename(
    columns={
        '2022   ': 'SUMA_TURISMO_T_1',
        '2023  ': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Cerrado
df_cerrado['TIPO_PERIODO'] = 'TURISMO-CERRADO'


# Corrido
df_control_departamentos_corrido = df_control_departamentos[
    (df_control_departamentos['Nombres de medidas'] == 'YTD anterior') | 
    (df_control_departamentos['Nombres de medidas'] == 'YTD actual')]

# Pivot
df_corrido = pd.DataFrame(df_control_departamentos_corrido.pivot_table(
    index=['ID_DPTO', 'Departamento'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_corrido.columns.name = None

# Renombrar columnas para el caso Corrido
df_corrido.rename(
    columns={
        'YTD anterior': 'SUMA_TURISMO_T_1',
        'YTD actual': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Corrido
df_corrido['TIPO_PERIODO'] = 'TURISMO-CORRIDO'

# Concatenar ambos df de control
df_control_departamentos = pd.concat([df_cerrado, df_corrido])

# Asegurar que los valores en DPTO_HOSPEDAJE son strings y rellenar con ceros a la izquierda si tienen longitud menor a 2
df_control_departamentos['ID_DPTO'] = df_control_departamentos['ID_DPTO'].astype(str).str.zfill(2)

# Agregar agrupación
df_control_departamentos['AGRUPACION'] = 'DEPARTAMENTOS'

In [31]:
# Obtener correlativa de departamentos
df_dept = session.table('DOCUMENTOS_COLOMBIA.GEOGRAFIA.DIAN_DEPARTAMENTOS').select(
            'COD_DIAN_DEPARTAMENTO',  # Código del departamento según DIAN
            'DEPARTAMENTO_DIAN'       # Nombre del departamento según DIAN
        ).to_pandas()

In [32]:
# Agregar nombre de DEPARTAMENTO_DIAN
df_control_departamentos = df_control_departamentos.merge(df_dept, how='left', left_on='ID_DPTO', right_on='COD_DIAN_DEPARTAMENTO')

# Cambiar nombres para seguir con la estructura
df_control_departamentos = df_control_departamentos.rename(columns={'DEPARTAMENTO_DIAN' : 'UNIDAD'})

# Elegir columnas relevantes
df_control_departamentos = df_control_departamentos[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

# Df de control
ST_CONTROL_DEPARTAMENTOS = df_control_departamentos.dropna(subset=['UNIDAD'])
ST_CONTROL_DEPARTAMENTOS = ST_CONTROL_DEPARTAMENTOS.groupby(['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO'])[['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']].sum().reset_index() 

In [33]:
########
# PAISES
########

# Archivo paises
pais_control = 'PAISES_data.csv'
df_control_pais = pd.read_csv(path_turismo + pais_control, sep=";", decimal=",", dtype={'COD_TURISMO (DIM_PAIS) #2':str, 'País residencia':str})

# Elegir años de interés

# Cerrado
df_control_pais_cerrado = df_control_pais[
    (df_control_pais['Nombres de medidas'] == '2022   ') | 
    (df_control_pais['Nombres de medidas'] == '2023  ') ]

# Pivot
df_cerrado = pd.DataFrame(df_control_pais_cerrado.pivot_table(
    index=['COD_TURISMO (DIM_PAIS) #2', 'País residencia'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_cerrado.columns.name = None

# Renombrar columnas para el caso Cerrado
df_cerrado.rename(
    columns={
        '2022   ': 'SUMA_TURISMO_T_1',
        '2023  ': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Cerrado
df_cerrado['TIPO_PERIODO'] = 'TURISMO-CERRADO'


# Corrido
df_control_pais_corrido = df_control_pais[
    (df_control_pais['Nombres de medidas'] == 'YTD anterior') | 
    (df_control_pais['Nombres de medidas'] == 'YTD actual')]

# Pivot
df_corrido = pd.DataFrame(df_control_pais_corrido.pivot_table(
    index=['COD_TURISMO (DIM_PAIS) #2', 'País residencia'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_corrido.columns.name = None

# Renombrar columnas para el caso Corrido
df_corrido.rename(
    columns={
        'YTD anterior': 'SUMA_TURISMO_T_1',
        'YTD actual': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Corrido
df_corrido['TIPO_PERIODO'] = 'TURISMO-CORRIDO'

# Concatenar ambos df de control
df_control_pais = pd.concat([df_cerrado, df_corrido])

# Agregar agrupación
df_control_pais['AGRUPACION'] = 'PAISES'

# Guardar insumo de tlcs para paso posterior
df_insumo_tcls = df_control_pais

In [34]:
# Obtener correlativa de países
df_paises = pd.DataFrame(session.sql("""
SELECT DISTINCT A.CODIGO_PAIS_MIGRACION, 
    A.PAIS_LLAVE_EXPORTACIONES
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A
""").collect())

In [35]:
# Agregar nombre de PAIS_LLAVE_EXPORTACIONES
df_control_pais = df_control_pais.merge(df_paises, how='left', left_on='COD_TURISMO (DIM_PAIS) #2', right_on='CODIGO_PAIS_MIGRACION')

# Cambiar nombres para seguir con la estructura
df_control_pais = df_control_pais.rename(columns={'PAIS_LLAVE_EXPORTACIONES' : 'UNIDAD'})

# Elegir columnas relevantes
df_control_pais = df_control_pais[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

# Df de control
ST_CONTROL_PAIS = df_control_pais.dropna(subset=['UNIDAD'])
ST_CONTROL_PAIS = ST_CONTROL_PAIS.groupby(['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO'])[['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']].sum().reset_index() 

In [36]:
#############
# CONTINENTES
#############

# Archivo continentes
continentes_control = 'CONTINENTES-TOTALES_data.csv'
df_control_continente = pd.read_csv(path_turismo + continentes_control, sep=";", decimal=",", dtype={'Región país residencia':str})

# Elegir años de interés

# Cerrado
df_control_continente_cerrado = df_control_continente[
    (df_control_continente['Nombres de medidas'] == '2022   ') | 
    (df_control_continente['Nombres de medidas'] == '2023  ') ]

# Pivot
df_cerrado = pd.DataFrame(df_control_continente_cerrado.pivot_table(
    index=['Región país residencia'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_cerrado.columns.name = None

# Renombrar columnas para el caso Cerrado
df_cerrado.rename(
    columns={
        '2022   ': 'SUMA_TURISMO_T_1',
        '2023  ': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Cerrado
df_cerrado['TIPO_PERIODO'] = 'TURISMO-CERRADO'


# Corrido
df_control_continente_corrido = df_control_continente[
    (df_control_continente['Nombres de medidas'] == 'YTD anterior') | 
    (df_control_continente['Nombres de medidas'] == 'YTD actual')]

# Pivot
df_corrido = pd.DataFrame(df_control_continente_corrido.pivot_table(
    index=['Región país residencia'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_corrido.columns.name = None

# Renombrar columnas para el caso Corrido
df_corrido.rename(
    columns={
        'YTD anterior': 'SUMA_TURISMO_T_1',
        'YTD actual': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Corrido
df_corrido['TIPO_PERIODO'] = 'TURISMO-CORRIDO'

# Concatenar ambos df de control
df_control_continente = pd.concat([df_cerrado, df_corrido])

# Agregar agrupación
df_control_continente['AGRUPACION'] = 'CONTINENTES'

In [37]:
# Obtener correlativa de continentes
df_continentes = pd.DataFrame(session.sql("""
SELECT DISTINCT A.REGION_NAME_TURISMO, 
    B.REGION_NAME_EXPORTACIONES
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.PAISES_TURISMO AS A
LEFT JOIN DOCUMENTOS_COLOMBIA.GEOGRAFIA.CONTINENTES AS B ON A.REGION_NAME_TURISMO_AGREGADA = B.REGION_NAME_TURISMO_AGREGADA;
""").collect())

In [38]:
# Agregar nombre de REGION_NAME_EXPORTACIONES
df_control_continente = df_control_continente.merge(df_continentes, how='left', left_on='Región país residencia', right_on='REGION_NAME_TURISMO')

# Cambiar nombres para seguir con la estructura
df_control_continente = df_control_continente.rename(columns={'REGION_NAME_EXPORTACIONES' : 'UNIDAD'})

# Elegir columnas relevantes
df_control_continente = df_control_continente[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

# Df de control
ST_CONTROL_CONTINENTES = df_control_continente.dropna(subset=['UNIDAD'])
ST_CONTROL_CONTINENTES = ST_CONTROL_CONTINENTES.groupby(['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO'])[['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']].sum().reset_index() 

In [40]:
# Modificar los datos para asignar Egipto al total de África y eliminarlo de Asia. 

# Filtrar los datos de Egipto
df_egipto = ST_CONTROL_PAIS[ST_CONTROL_PAIS['UNIDAD'] == 'Egipto']

# Iterar por cada TIPO_PERIODO único
for tipo_periodo in df_egipto['TIPO_PERIODO'].unique():
    # Valores de Egipto para el periodo actual
    suma_t_egipto = df_egipto[df_egipto['TIPO_PERIODO'] == tipo_periodo]['SUMA_TURISMO_T'].values[0]
    suma_t_1_egipto = df_egipto[df_egipto['TIPO_PERIODO'] == tipo_periodo]['SUMA_TURISMO_T_1'].values[0]

    # Modificación para Asia (restar valores)
    ST_CONTROL_CONTINENTES.loc[
        (ST_CONTROL_CONTINENTES['UNIDAD'] == 'Asia') & (ST_CONTROL_CONTINENTES['TIPO_PERIODO'] == tipo_periodo),
        ['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']
    ] -= [suma_t_egipto, suma_t_1_egipto]

    # Modificación para África (sumar valores)
    ST_CONTROL_CONTINENTES.loc[
        (ST_CONTROL_CONTINENTES['UNIDAD'] == 'África') & (ST_CONTROL_CONTINENTES['TIPO_PERIODO'] == tipo_periodo),
        ['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']
    ] += [suma_t_egipto, suma_t_1_egipto]

In [41]:
######
# HUBS
######

# Archivo continentes
hubs_control = 'HUB-TOTALES_data.csv'
df_control_hubs = pd.read_csv(path_turismo + hubs_control, sep=";", decimal=",", dtype={'HUB__C (País residencia) (grupo)' : str})

# Elegir años de interés

# Cerrado
df_control_hubs_cerrado = df_control_hubs[
    (df_control_hubs['Nombres de medidas'] == '2022   ') | 
    (df_control_hubs['Nombres de medidas'] == '2023  ') ]

# Pivot
df_cerrado = pd.DataFrame(df_control_hubs_cerrado.pivot_table(
    index=['HUB__C (País residencia) (grupo)'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_cerrado.columns.name = None

# Renombrar columnas para el caso Cerrado
df_cerrado.rename(
    columns={
        '2022   ': 'SUMA_TURISMO_T_1',
        '2023  ': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Cerrado
df_cerrado['TIPO_PERIODO'] = 'TURISMO-CERRADO'


# Corrido
df_control_hubs_corrido = df_control_hubs[
    (df_control_hubs['Nombres de medidas'] == 'YTD anterior') | 
    (df_control_hubs['Nombres de medidas'] == 'YTD actual')]

# Pivot
df_corrido = pd.DataFrame(df_control_hubs_corrido.pivot_table(
    index=['HUB__C (País residencia) (grupo)'],
    columns='Nombres de medidas',
    values='Valores de medidas',
    aggfunc='sum'
)).reset_index()

# Eliminar el índice de nombre en las columnas
df_corrido.columns.name = None

# Renombrar columnas para el caso Corrido
df_corrido.rename(
    columns={
        'YTD anterior': 'SUMA_TURISMO_T_1',
        'YTD actual': 'SUMA_TURISMO_T'
    },
    inplace=True
)

# Corrido
df_corrido['TIPO_PERIODO'] = 'TURISMO-CORRIDO'

# Concatenar ambos df de control
df_control_hubs = pd.concat([df_cerrado, df_corrido])

# Agregar agrupación
df_control_hubs['AGRUPACION'] = 'HUBS'

In [42]:
# Obtener correlativa de hubs
df_hubs = pd.DataFrame(session.sql("""
SELECT DISTINCT A.HUB_NAME_EXPORTACIONES, 
    A.HUB_NAME_TURISMO
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.HUBS AS A;
""").collect())

In [43]:
# Agregar nombre de HUB_NAME_EXPORTACIONES
df_control_hubs = df_control_hubs.merge(df_hubs, how='left', left_on='HUB__C (País residencia) (grupo)', right_on='HUB_NAME_TURISMO')

# Cambiar nombres para seguir con la estructura
df_control_hubs = df_control_hubs.rename(columns={'HUB_NAME_EXPORTACIONES' : 'UNIDAD'})

# Elegir columnas relevantes
df_control_hubs = df_control_hubs[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

# Df de control
ST_CONTROL_HUBS = df_control_hubs.dropna(subset=['UNIDAD'])
ST_CONTROL_HUBS = ST_CONTROL_HUBS.groupby(['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO'])[['SUMA_TURISMO_T', 'SUMA_TURISMO_T_1']].sum().reset_index() 

In [44]:
######
# TLCS
######

# Correlativa de TLCs
df_tlcs = pd.DataFrame(session.sql("""
SELECT DISTINCT A.CODIGO_PAIS_MIGRACION,       
    A.NOMBRE_TLC
FROM DOCUMENTOS_COLOMBIA.GEOGRAFIA.ST_PAISES AS A;
""").collect())

# Agregar nombre de NOMBRE_TLC
df_insumo_tcls = df_insumo_tcls.merge(df_tlcs, how='left', left_on='COD_TURISMO (DIM_PAIS) #2', right_on='CODIGO_PAIS_MIGRACION')

# Cambiar nombres para seguir con la estructura
df_insumo_tcls = df_insumo_tcls.rename(columns={'NOMBRE_TLC' : 'UNIDAD'})

# Elegir columnas relevantes
df_insumo_tcls = df_insumo_tcls[['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO', 'SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']]

# Agrupación
df_insumo_tcls['AGRUPACION'] = 'TLCS'

# Df de control
ST_CONTROL_TLCS = df_insumo_tcls.dropna(subset=['UNIDAD'])
ST_CONTROL_TLCS = ST_CONTROL_TLCS.groupby(['AGRUPACION', 'UNIDAD', 'TIPO_PERIODO'])[['SUMA_TURISMO_T_1', 'SUMA_TURISMO_T']].sum().reset_index() 

In [45]:
# Base de control total
DF_TURISMO_CONTROL = pd.concat([ST_CONTROL_COLOMBIA, ST_CONTROL_DEPARTAMENTOS, ST_CONTROL_PAIS, ST_CONTROL_CONTINENTES, ST_CONTROL_HUBS, ST_CONTROL_TLCS])

In [46]:
# Realizar cambio de continente agregado a Egipto
DF_TURISMO_CONTROL[DF_TURISMO_CONTROL['AGRUPACION']=='CONTINENTES']

,AGRUPACION,UNIDAD,TIPO_PERIODO,SUMA_TURISMO_T_1,SUMA_TURISMO_T
0,CONTINENTES,América,TURISMO-CERRADO,"2,698,372.0000000000","3,351,855.0000000000"
1,CONTINENTES,América,TURISMO-CORRIDO,"2,737,199.0000000000","3,049,085.0000000000"
2,CONTINENTES,Asia,TURISMO-CERRADO,"40,095.0000000000","67,547.0000000000"
3,CONTINENTES,Asia,TURISMO-CORRIDO,"55,418.0000000000","65,655.0000000000"
4,CONTINENTES,Europa,TURISMO-CERRADO,"521,721.0000000000","630,399.0000000000"
5,CONTINENTES,Europa,TURISMO-CORRIDO,"510,580.0000000000","525,323.0000000000"
6,CONTINENTES,Oceanía,TURISMO-CERRADO,"13,354.0000000000","18,708.0000000000"
7,CONTINENTES,Oceanía,TURISMO-CORRIDO,"14,218.0000000000","15,286.0000000000"
8,CONTINENTES,África,TURISMO-CERRADO,"5,158.0000000000","6,121.0000000000"
9,CONTINENTES,África,TURISMO-CORRIDO,"4,876.0000000000","6,176.0000000000"


## 5. Subir a Snowflake

##### Seleccionar base de datos y esquema

In [47]:
# Usar base de datos:
sql_database_usar = """
USE DOCUMENTOS_COLOMBIA;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_database_usar)

,status
0,Statement executed successfully.


In [48]:
# Usar esquema turismo:
sql_schema_turismo = """
USE SCHEMA TURISMO;
"""
# Ejecutar
snow_func.snowflake_sql(conn, sql_schema_turismo)

,status
0,Statement executed successfully.


In [49]:
# Asegurar que estamos en la ubicación que se desea para subir las bases de datos
ubicacion = "SELECT CURRENT_WAREHOUSE() AS WAREHOUSE, CURRENT_DATABASE() AS DATABASE, CURRENT_SCHEMA() AS SCHEMA;"
# Ejecutar
snow_func.snowflake_sql(conn, ubicacion)

,WAREHOUSE,DATABASE,SCHEMA
0,WH_PROCOLOMBIA_ANALITICA,DOCUMENTOS_COLOMBIA,TURISMO


In [50]:
# Lista de pd a subir
bases_de_datos = [
    ST_PAISES_CERRADO,
    ST_PAISES_CORRIDO,
    df_conectividad,
    DF_TURISMO_CONTROL
]

nombres_tablas = [
    'ST_PAISES_CERRADO',
    'ST_PAISES_CORRIDO',
    'CONECTIVIDAD',
    'DF_TURISMO_CONTROL'
]

In [51]:
# pd de verificación
df_resultados_verificacion = pd.DataFrame()
# Subir y verificar bases a Snowflake
for base, tabla in zip(bases_de_datos, nombres_tablas):
    # Cargar el DataFrame en Snowflake y capturar el mensaje de carga
    mensaje_carga = snow_func.snowflake_cargar_df(conn, base, f'{tabla}')
    
    # Verificar y almacenar el resultado en el DataFrame
    resultado = snow_func.snowflake_sql(conn, f"SELECT COUNT(*) FROM {tabla};")
    total_registros = resultado  # Extraer el total de registros

    # Crear un DataFrame temporal para la nueva fila
    nueva_fila = pd.DataFrame({
        'Tabla': [tabla],
        'Total_Registros': [total_registros],
        'Mensaje_Carga': [mensaje_carga]
    })

    # Concatenar la nueva fila al DataFrame de resultados
    df_resultados_verificacion = pd.concat([df_resultados_verificacion, nueva_fila], ignore_index=True)

In [52]:
# Ver resultados
df_resultados_verificacion

,Tabla,Total_Registros,Mensaje_Carga
0,ST_PAISES_CERRADO,COUNT(*) 0 147262,DataFrame cargado exitosamente en la tabla: 147262 filas en 1 chunks.\nTiempo de carga: 4.95 segundos.\nProceso terminado
1,ST_PAISES_CORRIDO,COUNT(*) 0 137132,DataFrame cargado exitosamente en la tabla: 137132 filas en 1 chunks.\nTiempo de carga: 3.91 segundos.\nProceso terminado
2,CONECTIVIDAD,COUNT(*) 0 404,DataFrame cargado exitosamente en la tabla: 404 filas en 1 chunks.\nTiempo de carga: 2.59 segundos.\nProceso terminado
3,DF_TURISMO_CONTROL,COUNT(*) 0 574,DataFrame cargado exitosamente en la tabla: 574 filas en 1 chunks.\nTiempo de carga: 1.70 segundos.\nProceso terminado


### Cerrar sesión, conexión y cursor

In [53]:
conn.close()
session.close()